In [1]:
# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import unicodedata
import re
import numpy as np
import pandas as pd
import os
import io
import time
import pickle
from sklearn.model_selection import train_test_split
import string
from string import digits
import re
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Input, Dense,Embedding, Concatenate, TimeDistributed
from tensorflow.keras.models import Model,load_model, model_from_json
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K 
import pickle as pkl
import numpy as np

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from string import digits
import re
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Input, Dense,Embedding, Concatenate, TimeDistributed
from tensorflow.keras.models import Model,load_model, model_from_json
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K 
import pickle as pkl
import numpy as np

In [3]:
import tensorflow as tf
from tensorflow.python.keras import backend as K

logger = tf.get_logger()

class AttentionLayer(tf.keras.layers.Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs

        logger.debug(f"encoder_out_seq.shape = {encoder_out_seq.shape}")
        logger.debug(f"decoder_out_seq.shape = {decoder_out_seq.shape}")

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            logger.debug("Running energy computation step")

            if not isinstance(states, (list, tuple)):
                raise TypeError(f"States must be an iterable. Got {states} of type {type(states)}")

            encoder_full_seq = states[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_full_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim

            logger.debug(f"U_a_dot_h.shape = {U_a_dot_h.shape}")

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)

            logger.debug(f"Ws_plus_Uh.shape = {Ws_plus_Uh.shape}")

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            logger.debug(f"ei.shape = {e_i.shape}")

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            logger.debug("Running attention vector computation step")

            if not isinstance(states, (list, tuple)):
                raise TypeError(f"States must be an iterable. Got {states} of type {type(states)}")

            encoder_full_seq = states[-1]

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_full_seq * K.expand_dims(inputs, -1), axis=1)

            logger.debug(f"ci.shape = {c_i.shape}")

            return c_i, [c_i]

        # we don't maintain states between steps when computing attention
        # attention is stateless, so we're passing a fake state for RNN step function
        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e], constants=[encoder_out_seq]
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c], constants=[encoder_out_seq]
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [4]:
json_file = open('Spell_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_loaded = model_from_json(loaded_model_json, custom_objects={'AttentionLayer': AttentionLayer})
# load weights into new model
model_loaded.load_weights("Spell_model_weight.h5")

In [5]:
with open('inputTokenizer.pkl', 'rb') as f:
    inputTokenizer = pickle.load(f)

In [6]:
with open('outputTokenizer.pkl', 'rb') as f:
    outputTokenizer = pickle.load(f)

In [7]:
Eindex2word = inputTokenizer.index_word

In [8]:
Mindex2word = outputTokenizer.index_word

In [9]:
latent_dim=300
# encoder inference
encoder_inputs = model_loaded.input[0]  #loading encoder_inputs
encoder_outputs, state_h, state_c = model_loaded.layers[4].output #loading encoder_outputs

print(encoder_outputs.shape)

encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# decoder inference
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,), name="decoder_state_input_h")
decoder_state_input_c = Input(shape=(latent_dim,), name="decoder_state_input_c")
decoder_hidden_state_input = Input(shape=(34,latent_dim))

# Get the embeddings of the decoder sequence
decoder_inputs = model_loaded.layers[1].output

print(decoder_inputs.shape)
dec_emb_layer = model_loaded.layers[3]

dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_lstm = model_loaded.layers[5]
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_layer = model_loaded.layers[6]
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])

concate = model_loaded.layers[7]
decoder_inf_concat = concate([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_dense = model_loaded.layers[8]
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])


(None, 34, 300)
(None, None)


In [10]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = Mword2index['<']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c], verbose=0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 0:
          break
        else:
          sampled_token = Mindex2word[sampled_token_index]

          if(sampled_token!='>'):
              decoded_sentence += ''+sampled_token

              # Exit condition: either hit max length or find stop word.
              if (sampled_token == '>' or len(decoded_sentence.split()) >= (26-1)):
                  stop_condition = True

          # Update the target sequence (of length 1).
          target_seq = np.zeros((1,1))
          target_seq[0, 0] = sampled_token_index

          # Update internal states
          e_h, e_c = h, c

    return decoded_sentence

In [11]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
      if((i!=0 and i!=Mword2index['<']) and i!=Mword2index['>']):
        newString=newString+Mindex2word[i]+''
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+Eindex2word[i]+''
    return newString

In [12]:
test1 = "emekdaw ile daniwdim dedi 4 iw gununde arawdiralacaq"
test2 = "ceta yazdi yuklenmedi indi baxiramki pul cixib."
test3 = "Hec bir banka odenisim yoxdur"
test4 = "her bir yerinde gul cicekle bayram"
olmayan1 = "en gozel ders bu dersdir"
olmayan2 = "yaxin adam orani qutarib"
olmayan3 = "gordum seni eve gel"
olmayan4 = "calis eve tez gel"
olmayan5 = "meni ora getirme"
olmayan6 = "basqa sozun var mene"
olmayan7 = "o ozu bilir"
olmayan8 = "inanmiram cox islesin"
olmayan9 = "mence birazdan gelecek"
olmayan10 = "bu qeder cumle besdi"

#olmayan1 = "salam. beynelxalqbankin kartina pul kocurmek isteyrem 200 manatdan artıq kocmur gun erzinde"

In [13]:
Mword2index = outputTokenizer.word_index

In [15]:
given = olmayan8
new_sample = ["< " + given +" >"]
new_sample = np.array(pad_sequences(inputTokenizer.texts_to_sequences(new_sample), maxlen=34, padding='post'))
print("Predicted summary:", decode_sequence(new_sample.reshape(1, 34)))

Predicted summary:  inanmıram çox işləsin 


In [ ]:
#predicting 100 unseen sentences

import pandas as pd

# read in the dataset from an Excel file
df = pd.read_csv('C:/Users/99451/Desktop/sdp_sentences/test_dataset_100_csv.csv')

# initialize an empty list to store the predicted summaries
summaries = []

# loop over each sentence in the dataset
for sentence in df['W']:
    # preprocess the sentence
    sentence = "< " + sentence + " >"
    sentence = np.array(pad_sequences(inputTokenizer.texts_to_sequences([sentence]), maxlen=34, padding='post'))

    # generate the summary for the sentence
    summary = decode_sequence(sentence.reshape(1, 34))
    summaries.append(summary)

# add the predicted summaries to the DataFrame
df['predicted_summary'] = summaries

# save the DataFrame to a new CSV file

df.to_csv('test_dataset_100_with_summaries.csv', index=False)


In [16]:
#accuracy of predicted true words divided max possibble expected true words

# import the necessary libraries
import pandas as pd
import Levenshtein

# read in the dataset from a CSV file
df = pd.read_csv('C:/Users/99451/Desktop/sdp_sentences/test_dataset_100_with_summaries_csv.csv')

# initialize a variable to store the total number of sentences
total_sentences = len(df)

# initialize a variable to store the total number of matched words
total_matched_words = 0

# initialize a variable to store the total number of possible matched words
total_possible_matched_words = 0

# loop over each row in the dataset
for index, row in df.iterrows():
    # get the true sentence and predicted sentence from the row
    true_sentence = row['C']
    predicted_sentence = row['predicted_summary']
    
    # split the sentences into words
    true_words = true_sentence.split()
    predicted_words = predicted_sentence.split()
    
    # initialize a variable to store the number of matched words in this sentence
    matched_words = 0
    
    # loop over each word in the true sentence
    for true_word in true_words:
        # loop over each word in the predicted sentence
        for predicted_word in predicted_words:
            # calculate the Levenshtein distance between the true and predicted words
            distance = Levenshtein.distance(true_word, predicted_word)
            
            # if the distance is 0, the words match
            if distance == 0:
                # increment the number of matched words and break out of the inner loop
                matched_words += 1
                break
                
    # add the number of matched words to the total number of matched words
    total_matched_words += matched_words
    
    # add the length of the true sentence to the total possible matched words
    total_possible_matched_words += len(true_words)

# calculate the overall matching percentage as a percentage
matching_percentage = (total_matched_words / total_possible_matched_words) * 100

# print the matching percentage
print(f"Matching percentage: {matching_percentage:.2f}%")


Matching percentage: 73.51%


In [17]:
#accuracy of expected true and predicted true word by word matching 
# import the necessary libraries
import pandas as pd
import Levenshtein

# read in the dataset from a CSV file
df = pd.read_csv('C:/Users/99451/Desktop/sdp_sentences/test_dataset_100_with_summaries_csv.csv')

# initialize a variable to store the total number of sentences
total_sentences = len(df)

# initialize a variable to store the total number of matching words
total_matching_words = 0

# initialize a variable to store the total number of words in the true sentences
total_true_words = 0

# loop over each row in the dataset
for index, row in df.iterrows():
    # get the true sentence and predicted sentence from the row
    true_sentence = row['C']
    predicted_sentence = row['predicted_summary']
    
    # split the sentences into words
    true_words = true_sentence.split()
    predicted_words = predicted_sentence.split()
    
    # calculate the number of matching words
    matching_words = sum(1 for x, y in zip(true_words, predicted_words) if x == y)
    
    # update the total number of matching words and true words
    total_matching_words += matching_words
    total_true_words += len(true_words)

# calculate the word-by-word matching percentage
word_matching_percentage = (total_matching_words / total_true_words) * 100

# print the word-by-word matching percentage
print(f"Word-by-word matching percentage: {word_matching_percentage:.2f}%")


Word-by-word matching percentage: 72.32%


In [20]:
#CER Accuracy

def compute_cer(ref, hyp):
    """
    Compute Character Error Rate (CER) between reference and hypothesis strings.
    """
    # Initialize substitution, deletion and insertion counters to zero
    S, D, I = 0, 0, 0
    # Compute Levenshtein distance between reference and hypothesis strings
    L = [[0] * (len(hyp) + 1) for _ in range(len(ref) + 1)]
    for i in range(len(ref) + 1):
        L[i][0] = i
    for j in range(len(hyp) + 1):
        L[0][j] = j
    for i in range(1, len(ref) + 1):
        for j in range(1, len(hyp) + 1):
            cost = (ref[i-1] != hyp[j-1])
            L[i][j] = min(L[i-1][j] + 1, L[i][j-1] + 1, L[i-1][j-1] + cost)
    # Compute S, D and I by backtracking the Levenshtein matrix
    i, j = len(ref), len(hyp)
    while i > 0 or j > 0:
        if i > 0 and j > 0 and ref[i-1] == hyp[j-1]:
            i, j = i - 1, j - 1
        elif i > 0 and L[i][j] == L[i-1][j] + 1:
            D += 1
            i = i - 1
        elif j > 0 and L[i][j] == L[i][j-1] + 1:
            I += 1
            j = j - 1
        else:
            S += 1
            i, j = i - 1, j - 1
    # Compute CER as the sum of S, D and I divided by the number of characters in the reference string
    cer = (S + D + I) / len(ref)
    return cer

In [22]:
#CER Accuracy
import pandas as pd

# Load test dataset as a pandas DataFrame with "W" and "C" columns
test_data = pd.read_csv('C:/Users/99451/Desktop/sdp_sentences/test_dataset_100_with_summaries_csv.csv')

# Compute CER for each example in the test dataset
cer_scores = []
for i in range(len(test_data)):
    ref = test_data.loc[i, "C"]
    hyp = test_data.loc[i, "predicted_summary"]
    cer = compute_cer(ref, hyp)
    cer_scores.append(cer)

# Compute the average CER score for the test dataset
average_cer = sum(cer_scores) / len(cer_scores)
print("Average CER: {:.2%}".format(average_cer))

Average CER: 13.84%
